### Adjacency method for different variables and different thresholds

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import itertools
from itertools import combinations
from scipy import spatial
import pickle as pickle
import gudhi
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import io
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageChops, ImageFont
import shapely.geometry as geom
from shapely.ops import unary_union
import warnings

import invr

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#### Creating the test dataset

In [3]:
# Create a DataFrame with box names and values
box_names = [chr(65 + i) for i in range(16)]
values = list(range(0, 16))
values_1 = [1,2,3,4,8,7,6,5,9,10,11,12,16,15,14,13]
values_3 = [1,5,3,16,14,2,15,4,7,13,8,9,12,10,11,6]
values_2 = list(range(1, 17))
df = pd.DataFrame({'FIPS': box_names, 'Value': values,'Value_1': values_1,'Value_2': values_2,'Value_3': values_3})

In [4]:
df

,FIPS,Value,Value_1,Value_2,Value_3
0,A,0,1,1,1
1,B,1,2,2,5
2,C,2,3,3,3
3,D,3,4,4,16
4,E,4,8,5,14
5,F,5,7,6,2
6,G,6,6,7,15
7,H,7,5,8,4
8,I,8,9,9,7
9,J,9,10,10,13


In [5]:
# Sorting the DataFrame based on the 'rate' column
df.sort_values(by='Value_2', inplace=True)

In [6]:
# Adding a new column 'new_ID' with ID values starting from zero
df['sortedID'] = range(len(df))

In [7]:
df = df[['FIPS','Value', 'sortedID', 'Value_2']]

In [8]:
# Function to calculate square coordinates for a given name
def calculate_square_coordinates(row):
    box_name = row['FIPS']
    value = row['Value']
    # Assuming each smaller square has a side length of 1 unit
    x = value % 4
    y = value // 4
    # Return square coordinates as a Shapely polygon
    return geom.Polygon([(x, y), (x+1, y), (x+1, y+1), (x, y+1)])

In [9]:
# Calculate square coordinates for each row and create a geometry column
df['geometry'] = df.apply(calculate_square_coordinates, axis=1)

/var/folders/j3/904q7dbj02j_jyjz7pvfgz384d40kw/T/ipykernel_18592/2838188260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(calculate_square_coordinates, axis=1)


In [10]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [11]:
# Set the CRS to a simple Cartesian coordinate system
gdf.crs = "EPSG:3395"  # This is a commonly used projected CRS

In [12]:
gdf.sort_values(by='Value', inplace=True)

In [13]:
gdf

,FIPS,Value,sortedID,Value_2,geometry
0,A,0,0,1,"POLYGON ((0.000 0.000, 1.000 0.000, 1.000 1.00..."
1,B,1,1,2,"POLYGON ((1.000 0.000, 2.000 0.000, 2.000 1.00..."
2,C,2,2,3,"POLYGON ((2.000 0.000, 3.000 0.000, 3.000 1.00..."
3,D,3,3,4,"POLYGON ((3.000 0.000, 4.000 0.000, 4.000 1.00..."
4,E,4,4,5,"POLYGON ((0.000 1.000, 1.000 1.000, 1.000 2.00..."
5,F,5,5,6,"POLYGON ((1.000 1.000, 2.000 1.000, 2.000 2.00..."
6,G,6,6,7,"POLYGON ((2.000 1.000, 3.000 1.000, 3.000 2.00..."
7,H,7,7,8,"POLYGON ((3.000 1.000, 4.000 1.000, 4.000 2.00..."
8,I,8,8,9,"POLYGON ((0.000 2.000, 1.000 2.000, 1.000 3.00..."
9,J,9,9,10,"POLYGON ((1.000 2.000, 2.000 2.000, 2.000 3.00..."


##### Multiple variable code starts here

In [ ]:
# define a dictionary wich has name and a value
selectedVariablesandThreshold = {'Value': 6, 'Value_2': 11}